In [ ]:
# Install requirements
!pip install -q -U bitsandbytes git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets trl wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import torch
from datasets import Dataset
import wandb
import os
import json

In [ ]:
#Initialize Weights & Biases
wandb.init(project="medical-llm-finetuning")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rahul9664171057 (rahul9664171057-iit-roorkee) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Load model with QLoRA configuration
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
# Prepare model for QLoRA training
model = prepare_model_for_kbit_training(model)

In [ ]:
# Configure QLoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,204,288 || trainable%: 0.5196


In [ ]:
# Prepare dataset with correct formatting
def create_prompt(patient_query, doctor_response):
    return f"""You are a conversational AI assistant acting as a professionally worded "e-doctor." Your role is to provide preliminary, text-based guidance in response to patient symptom-related questions.

### Patient Query:
{patient_query}

### AI Doctor Response:
{doctor_response}

*Disclaimer: This response is generated for informational purposes only and should not be considered a substitute for professional medical advice, diagnosis, or treatment. Please consult a licensed healthcare provider for personalized care.*"""

# Load and format dataset
with open("/content/english-train.json", "r") as f:
    raw_data = json.load(f)

formatted_data = []
for entry in raw_data:
    if len(entry["utterances"]) >= 2:
        patient_utterance = entry["utterances"][0].replace("patient:", "").strip()
        doctor_response = entry["utterances"][1].replace("doctor:", "").strip()

        formatted_text = create_prompt(patient_utterance, doctor_response)
        formatted_data.append({"text": formatted_text})

dataset = Dataset.from_list(formatted_data)

In [ ]:
training_args = TrainingArguments(
    output_dir="e-doctor-finetuned",
    max_steps=60,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=10,
    save_strategy="epoch",
    report_to="wandb",

)


def formatting_func(example):
    return example["text"]

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    formatting_func=formatting_func,
    # max_seq_length=1024
)


Applying formatting function to train dataset:   0%|          | 0/482 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/482 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/482 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/482 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/482 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# Train and save
trainer.train()
trainer.model.save_pretrained("e-doctor-qlora-adapter")
tokenizer.save_pretrained("e-doctor-qlora-adapter")

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.362300
20,1.439200
30,1.323500
40,1.361000
50,1.383300
60,1.359000


('e-doctor-qlora-adapter/tokenizer_config.json',
 'e-doctor-qlora-adapter/special_tokens_map.json',
 'e-doctor-qlora-adapter/chat_template.jinja',
 'e-doctor-qlora-adapter/tokenizer.json')

In [ ]:
!zip -r e-doctor-qlora-adapter e-doctor-qlora-adapter


  adding: e-doctor-qlora-adapter/ (stored 0%)
  adding: e-doctor-qlora-adapter/tokenizer.json (deflated 85%)
  adding: e-doctor-qlora-adapter/chat_template.jinja (deflated 75%)
  adding: e-doctor-qlora-adapter/adapter_model.safetensors (deflated 7%)
  adding: e-doctor-qlora-adapter/adapter_config.json (deflated 56%)
  adding: e-doctor-qlora-adapter/tokenizer_config.json (deflated 96%)
  adding: e-doctor-qlora-adapter/special_tokens_map.json (deflated 65%)
  adding: e-doctor-qlora-adapter/README.md (deflated 66%)


In [ ]:
from google.colab import files
files.download("e-doctor-qlora-adapter.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
PROMPT_TEMPLATE = """You are a conversational AI assistant acting as a professionally worded "e-doctor." Your role is to provide preliminary, text-based guidance in response to patient symptom-related questions.

### Patient Query:
{question}

### AI Doctor Response:
"""


In [ ]:
def get_e_doctor_response(patient_query, model, tokenizer,
                         max_new_tokens=100, temperature=0.7):
    # Format the prompt with the patient query
    prompt = PROMPT_TEMPLATE.format(question=patient_query)
    # Tokenize and move to the model's device
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    # Generate the response
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature
        )
    # Decode and return only the generated response (after the prompt)
    full_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = full_output.split("### AI Doctor Response:")[-1].strip()
    return response


In [ ]:
patient_query = "i have tuberculosis what should i do?"
response = get_e_doctor_response(patient_query, model, tokenizer)
print("AI Doctor:", response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AI Doctor: in brief: treatment tuberculosis requires specific treatment. you can only be diagnosed by a doctor on video or phone (telephonically). a chest x-ray is needed. the treatment depends on the form of tuberculosis. would you like to video or text chat with me?

*Disclaimer: This response is generated for informational purposes only and should not be considered a substitute for professional medical advice, diagnosis, or treatment. Please consult a licensed healthcare provider for personalized care.*


In [ ]:
patient_query = "dry scratchy throat, very mild cough since 23 march but not improving. traveled to cape town via lanseria and ct airports 15-20 march. wife has sore throat, nasal congestion, lethargy & headaches since 21 march. should we see a doctor to be assessed?"

response = get_e_doctor_response(patient_query, model, tokenizer)
print("AI Doctor:", response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


AI Doctor: yes. if you have access to a general physician, discuss your symptoms and see if a chest x-ray or some other diagnostic testing would be helpful. if you don't have access to a general physician, call your local emergency services and discuss the situation. covid-19 symptoms can be mild or severe. it's essential to contact a doctor or other healthcare professional to be screened for covid-19 if you have symptoms.

*Disclaimer: This response is generated for informational purposes only and should not be considered
